# 🧪 Testing APIs de Marketing Multiidioma
## Casos de Uso: Hotel de Lujo + Relojería Barcelona

Este notebook prueba paso a paso:
1. **Configuración de OpenAI API**
2. **API 1: Generador de Marketing**
3. **API 2: Traductor Especializado**
4. **Integración Completa**
5. **Casos de Uso Reales**

---

## 🔧 1. Setup y Configuración

In [1]:
# Instalar dependencias si es necesario
# !pip install openai fastapi uvicorn python-dotenv pydantic httpx

import os
import sys
import asyncio
import json
from datetime import datetime
from typing import List, Dict

# Agregar directorio actual al path para imports
sys.path.append(os.getcwd())

print("✅ Dependencias cargadas")
print(f"📍 Directorio actual: {os.getcwd()}")

✅ Dependencias cargadas
📍 Directorio actual: /Users/gabrielnoguera/Documents/BrainCode/CADARSO


In [2]:
# Configurar variables de entorno
from dotenv import load_dotenv

# Cargar archivo .env
load_dotenv()

# Verificar configuración
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

if not OPENAI_API_KEY or OPENAI_API_KEY == "your-openai-api-key-here":
    print("⚠️  IMPORTANTE: Configura tu OPENAI_API_KEY en el archivo .env")
    print("   Edita el archivo .env y reemplaza 'your-openai-api-key-here' con tu API key real")
else:
    print(f"✅ OpenAI API Key configurada: {OPENAI_API_KEY[:10]}...")
    print(f"🤖 Modelo: {OPENAI_MODEL}")

✅ OpenAI API Key configurada: sk-proj-hv...
🤖 Modelo: gpt-4o-mini


## 🧪 2. Test Básico de Conexión OpenAI

In [3]:
from openai import OpenAI

# Crear cliente OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Test básico de conexión
try:
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "Eres un asistente de prueba."},
            {"role": "user", "content": "Di 'Conexión exitosa' en español"}
        ],
        max_tokens=50
    )
    
    result = response.choices[0].message.content
    print(f"🎉 {result}")
    print(f"📊 Tokens usados: {response.usage.total_tokens}")
    
except Exception as e:
    print(f"❌ Error de conexión: {str(e)}")
    print("   Verifica tu API key y conexión a internet")

🎉 Conexión exitosa.
📊 Tokens usados: 35


## 📝 3. Modelos de Datos
Definimos las estructuras de datos que usarán nuestras APIs

In [4]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

# Tipos de productos
class ProductType(str, Enum):
    HOTEL_SERVICE = "hotel_service"
    HOTEL_AMENITY = "hotel_amenity" 
    LUXURY_WATCH = "luxury_watch"
    JEWELRY = "jewelry"
    SPA_SERVICE = "spa_service"

# Especificaciones del producto
class ProductSpecs(BaseModel):
    name: str
    type: ProductType
    category: str
    features: List[str]
    materials: Optional[List[str]] = None
    price_range: Optional[str] = None
    target_audience: str
    unique_selling_points: List[str]

# Request para marketing
class MarketingRequest(BaseModel):
    product: ProductSpecs
    style: str = "luxury"
    tone: str = "elegant"
    max_words: int = 150

# Idiomas soportados
class TargetLanguage(str, Enum):
    ENGLISH = "en"
    FRENCH = "fr"
    ITALIAN = "it"
    GERMAN = "de"
    PORTUGUESE = "pt"
    CATALAN = "ca"

print("✅ Modelos de datos definidos")
print(f"📦 Tipos de producto: {list(ProductType)}")
print(f"🌍 Idiomas soportados: {list(TargetLanguage)}")

✅ Modelos de datos definidos
📦 Tipos de producto: [<ProductType.HOTEL_SERVICE: 'hotel_service'>, <ProductType.HOTEL_AMENITY: 'hotel_amenity'>, <ProductType.LUXURY_WATCH: 'luxury_watch'>, <ProductType.JEWELRY: 'jewelry'>, <ProductType.SPA_SERVICE: 'spa_service'>]
🌍 Idiomas soportados: [<TargetLanguage.ENGLISH: 'en'>, <TargetLanguage.FRENCH: 'fr'>, <TargetLanguage.ITALIAN: 'it'>, <TargetLanguage.GERMAN: 'de'>, <TargetLanguage.PORTUGUESE: 'pt'>, <TargetLanguage.CATALAN: 'ca'>]


## 🎨 4. Test API 1: Generador de Marketing
Probamos la generación de contenido de marketing especializado

In [5]:
# Función simplificada para generar marketing
async def generate_marketing_content(product_specs: ProductSpecs) -> Dict:
    """Genera contenido de marketing usando OpenAI"""
    
    system_prompt = f"""
    Eres un experto en marketing de lujo especializado en {product_specs.type.value}.
    
    Crea contenido de marketing elegante y persuasivo.
    
    RESPONDE EN FORMATO JSON:
    {{
        "title": "Título atractivo del producto",
        "description": "Descripción de marketing (máximo 150 palabras)",
        "key_benefits": ["Beneficio 1", "Beneficio 2", "Beneficio 3"],
        "call_to_action": "Llamada a la acción específica",
        "seo_keywords": ["keyword1", "keyword2", "keyword3", "keyword4", "keyword5"]
    }}
    """
    
    features_text = ", ".join(product_specs.features)
    usp_text = ", ".join(product_specs.unique_selling_points)
    materials_text = ", ".join(product_specs.materials) if product_specs.materials else "No especificado"
    
    user_prompt = f"""
    PRODUCTO A PROMOCIONAR:
    Nombre: {product_specs.name}
    Tipo: {product_specs.type.value}
    Categoría: {product_specs.category}
    Características: {features_text}
    Materiales: {materials_text}
    Rango de precio: {product_specs.price_range or "Premium"}
    Público objetivo: {product_specs.target_audience}
    Puntos únicos de venta: {usp_text}
    
    Genera contenido de marketing profesional en JSON.
    """
    
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
            max_tokens=1500
        )
        
        raw_response = response.choices[0].message.content
        
        # Extraer JSON de la respuesta
        import re
        json_match = re.search(r'\{.*\}', raw_response, re.DOTALL)
        if json_match:
            json_str = json_match.group()
            return json.loads(json_str)
        else:
            raise ValueError("No se encontró JSON válido en la respuesta")
            
    except Exception as e:
        print(f"❌ Error generando marketing: {str(e)}")
        return None

print("✅ Función de marketing lista para probar")

✅ Función de marketing lista para probar


In [6]:
# Crear producto de prueba: Suite del hotel
suite_product = ProductSpecs(
    name="Suite Presidencial Barcelona",
    type=ProductType.HOTEL_SERVICE,
    category="Alojamiento Premium",
    features=[
        "Vista panorámica de la ciudad",
        "Terraza privada de 50m²", 
        "Servicio de mayordomo 24h",
        "Mobiliario de diseño exclusivo",
        "Sistema de domótica avanzado"
    ],
    price_range="€800-1200/noche",
    target_audience="Ejecutivos internacionales y viajeros de lujo",
    unique_selling_points=[
        "Única suite con terraza en el centro histórico",
        "Diseño personalizado por arquitecto reconocido",
        "Acceso VIP a servicios del hotel"
    ]
)

print("🏨 Producto de prueba creado:")
print(f"   📝 Nombre: {suite_product.name}")
print(f"   🎯 Tipo: {suite_product.type.value}")
print(f"   💰 Precio: {suite_product.price_range}")

🏨 Producto de prueba creado:
   📝 Nombre: Suite Presidencial Barcelona
   🎯 Tipo: hotel_service
   💰 Precio: €800-1200/noche


In [7]:
# Test API 1: Generar marketing para la suite
print("🎨 Generando contenido de marketing...")
print("⏳ Esto puede tardar 10-15 segundos...")

marketing_result = await generate_marketing_content(suite_product)

if marketing_result:
    print("\n" + "="*60)
    print("🎉 MARKETING GENERADO EXITOSAMENTE")
    print("="*60)
    
    print(f"\n📝 TÍTULO:")
    print(f"   {marketing_result['title']}")
    
    print(f"\n📄 DESCRIPCIÓN:")
    print(f"   {marketing_result['description']}")
    
    print(f"\n✨ BENEFICIOS CLAVE:")
    for i, benefit in enumerate(marketing_result['key_benefits'], 1):
        print(f"   {i}. {benefit}")
    
    print(f"\n🎯 CALL TO ACTION:")
    print(f"   {marketing_result['call_to_action']}")
    
    print(f"\n🔍 SEO KEYWORDS:")
    print(f"   {', '.join(marketing_result['seo_keywords'])}")
    
    # Contar palabras
    word_count = len(marketing_result['description'].split())
    print(f"\n📊 Estadísticas:")
    print(f"   Palabras en descripción: {word_count}")
    print(f"   Beneficios incluidos: {len(marketing_result['key_benefits'])}")
    print(f"   Keywords SEO: {len(marketing_result['seo_keywords'])}")
else:
    print("❌ Error generando marketing")

🎨 Generando contenido de marketing...
⏳ Esto puede tardar 10-15 segundos...

🎉 MARKETING GENERADO EXITOSAMENTE

📝 TÍTULO:
   Suite Presidencial Barcelona: Un Refugio de Lujo en el Corazón de la Ciudad

📄 DESCRIPCIÓN:
   Descubra la máxima expresión del lujo en la Suite Presidencial Barcelona, donde la elegancia se encuentra con la comodidad. Disfrute de impresionantes vistas panorámicas de la ciudad desde su terraza privada de 50m², diseñada para ofrecer un oasis de tranquilidad. Con un servicio de mayordomo las 24 horas y mobiliario de diseño exclusivo, cada detalle ha sido meticulosamente pensado para satisfacer las expectativas de los viajeros más exigentes. Experimente un estilo de vida sin igual mientras se sumerge en la sofisticación de esta suite única, que combina tecnología avanzada con un diseño personalizado por un arquitecto reconocido. Su estancia aquí es más que un alojamiento; es una experiencia inigualable.

✨ BENEFICIOS CLAVE:
   1. Terraza privada con vistas impresion

## 🌍 5. Test API 2: Traductor Especializado
Probamos la traducción cultural del contenido generado

In [8]:
# Función simplificada para traducir
async def translate_content(text: str, target_language: TargetLanguage, brand_name: str = "Luxury Hotel") -> Dict:
    """Traduce contenido con adaptación cultural"""
    
    language_names = {
        TargetLanguage.ENGLISH: "inglés",
        TargetLanguage.FRENCH: "francés", 
        TargetLanguage.ITALIAN: "italiano",
        TargetLanguage.GERMAN: "alemán",
        TargetLanguage.PORTUGUESE: "portugués",
        TargetLanguage.CATALAN: "catalán"
    }
    
    target_lang_name = language_names[target_language]
    
    system_prompt = f"""
    Eres un traductor profesional especializado en marketing de lujo para marcas internacionales.
    
    ESPECIALIZACIÓN:
    - Traducción de español a {target_lang_name}
    - Adaptación cultural para mercados de lujo
    - Mantenimiento del tono y estilo original
    
    FORMATO DE RESPUESTA (JSON):
    {{
        "translated_text": "Traducción completa y natural",
        "cultural_notes": ["Nota 1 sobre adaptación", "Nota 2"],
        "preserved_terms": ["término1", "término2"]
    }}
    """
    
    user_prompt = f"""
    TEXTO A TRADUCIR AL {target_lang_name.upper()}:
    "{text}"

    CONTEXTO:
    - Marca: {brand_name}
    - Tipo de contenido: Marketing de lujo hotelero
    - Adaptación cultural: Sí

    INSTRUCCIONES:
    1. Traduce manteniendo el impacto emocional
    2. Adapta culturalmente para el mercado {target_lang_name}
    3. Conserva el tono de lujo y exclusividad
    4. Proporciona notas sobre adaptaciones realizadas
    5. Responde SOLO en formato JSON válido
    
    ¡Traduce ahora!
    """
    
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # Más conservador para traducciones
            max_tokens=2000
        )
        
        raw_response = response.choices[0].message.content
        
        # Extraer JSON
        import re
        json_match = re.search(r'\{.*\}', raw_response, re.DOTALL)
        if json_match:
            json_str = json_match.group()
            result = json.loads(json_str)
            result['language'] = target_language
            return result
        else:
            # Fallback si no hay JSON válido
            return {
                "translated_text": raw_response.strip(),
                "cultural_notes": [],
                "preserved_terms": [],
                "language": target_language
            }
            
    except Exception as e:
        print(f"❌ Error traduciendo a {target_lang_name}: {str(e)}")
        return None

print("✅ Función de traducción lista para probar")

✅ Función de traducción lista para probar


In [9]:
# Test API 2: Traducir la descripción generada
if marketing_result:
    source_text = marketing_result['description']
    test_languages = [TargetLanguage.ENGLISH, TargetLanguage.FRENCH]
    
    print("🌍 Iniciando traducción a múltiples idiomas...")
    print(f"📝 Texto origen: {source_text[:100]}...")
    print(f"🎯 Idiomas objetivo: {[lang.value for lang in test_languages]}")
    
    translations = []
    
    for lang in test_languages:
        print(f"\n⏳ Traduciendo a {lang.value}...")
        
        translation = await translate_content(
            text=source_text,
            target_language=lang,
            brand_name="Hotel Barcelona Luxury"
        )
        
        if translation:
            translations.append(translation)
            print(f"   ✅ Traducción a {lang.value} completada")
        else:
            print(f"   ❌ Error traduciendo a {lang.value}")
    
    print(f"\n🎉 Traducciones completadas: {len(translations)}/{len(test_languages)}")
else:
    print("⚠️ No hay contenido de marketing para traducir. Ejecuta primero la celda anterior.")

🌍 Iniciando traducción a múltiples idiomas...
📝 Texto origen: Descubra la máxima expresión del lujo en la Suite Presidencial Barcelona, donde la elegancia se encu...
🎯 Idiomas objetivo: ['en', 'fr']

⏳ Traduciendo a en...
   ✅ Traducción a en completada

⏳ Traduciendo a fr...
   ✅ Traducción a fr completada

🎉 Traducciones completadas: 2/2


In [10]:
# Mostrar resultados de traducción
if 'translations' in locals() and translations:
    print("\n" + "="*80)
    print("🌍 RESULTADOS DE TRADUCCIÓN")
    print("="*80)
    
    print(f"\n📝 TEXTO ORIGINAL (Español):")
    print(f"   {source_text}")
    
    for translation in translations:
        lang_flag = {
            TargetLanguage.ENGLISH: "🇺🇸",
            TargetLanguage.FRENCH: "🇫🇷",
            TargetLanguage.ITALIAN: "🇮🇹",
            TargetLanguage.GERMAN: "🇩🇪",
            TargetLanguage.PORTUGUESE: "🇵🇹",
            TargetLanguage.CATALAN: "🏴"
        }.get(translation['language'], "🌍")
        
        print(f"\n{lang_flag} TRADUCCIÓN ({translation['language'].value.upper()}):")
        print(f"   {translation['translated_text']}")
        
        if translation['cultural_notes']:
            print(f"\n   📌 Notas culturales:")
            for note in translation['cultural_notes']:
                print(f"      • {note}")
        
        if translation['preserved_terms']:
            print(f"\n   🏷️ Términos preservados: {', '.join(translation['preserved_terms'])}")
    
    print(f"\n📊 Estadísticas:")
    print(f"   Idiomas traducidos: {len(translations)}")
    original_words = len(source_text.split())
    for translation in translations:
        translated_words = len(translation['translated_text'].split())
        ratio = (translated_words / original_words) * 100
        print(f"   {translation['language'].value}: {translated_words} palabras ({ratio:.1f}% del original)")
else:
    print("⚠️ No hay traducciones para mostrar")


🌍 RESULTADOS DE TRADUCCIÓN

📝 TEXTO ORIGINAL (Español):
   Descubra la máxima expresión del lujo en la Suite Presidencial Barcelona, donde la elegancia se encuentra con la comodidad. Disfrute de impresionantes vistas panorámicas de la ciudad desde su terraza privada de 50m², diseñada para ofrecer un oasis de tranquilidad. Con un servicio de mayordomo las 24 horas y mobiliario de diseño exclusivo, cada detalle ha sido meticulosamente pensado para satisfacer las expectativas de los viajeros más exigentes. Experimente un estilo de vida sin igual mientras se sumerge en la sofisticación de esta suite única, que combina tecnología avanzada con un diseño personalizado por un arquitecto reconocido. Su estancia aquí es más que un alojamiento; es una experiencia inigualable.

🇺🇸 TRADUCCIÓN (EN):
   Discover the ultimate expression of luxury in the Barcelona Presidential Suite, where elegance meets comfort. Enjoy breathtaking panoramic views of the city from your private 50m² terrace, designed t

## ⌚ 6. Test con Producto de Relojería
Probamos con un caso diferente para validar la especialización

In [ ]:
# Crear producto de relojería
watch_product = ProductSpecs(
    name="Rolex Submariner Edición Barcelona",
    type=ProductType.LUXURY_WATCH,
    category="Relojería Deportiva de Lujo",
    features=[
        "Movimiento automático suizo",
        "Resistente al agua hasta 300m",
        "Bisel giratorio unidireccional",
        "Cristal de zafiro antireflejo",
        "Brazalete Oyster de 3 eslabones"
    ],
    materials=["Acero inoxidable 904L", "Oro amarillo 18k", "Cristal de zafiro"],
    price_range="€15,000-25,000",
    target_audience="Coleccionistas y entusiastas de la relojería de lujo",
    unique_selling_points=[
        "Edición limitada exclusiva para Barcelona",
        "Grabado personalizado de la Sagrada Familia",
        "Certificado de autenticidad numerado",
        "Estuche de presentación artesanal"
    ]
)

print("⌚ Producto de relojería creado:")
print(f"   📝 Nombre: {watch_product.name}")
print(f"   🎯 Tipo: {watch_product.type.value}")
print(f"   💰 Precio: {watch_product.price_range}")
print(f"   🔧 Materiales: {', '.join(watch_product.materials)}")

In [ ]:
# Generar marketing para el reloj
print("⌚ Generando marketing para relojería de lujo...")
print("⏳ Procesando...")

watch_marketing = await generate_marketing_content(watch_product)

if watch_marketing:
    print("\n" + "="*60)
    print("⌚ MARKETING DE RELOJERÍA GENERADO")
    print("="*60)
    
    print(f"\n📝 TÍTULO:")
    print(f"   {watch_marketing['title']}")
    
    print(f"\n📄 DESCRIPCIÓN:")
    print(f"   {watch_marketing['description']}")
    
    print(f"\n✨ BENEFICIOS CLAVE:")
    for i, benefit in enumerate(watch_marketing['key_benefits'], 1):
        print(f"   {i}. {benefit}")
    
    print(f"\n🎯 CALL TO ACTION:")
    print(f"   {watch_marketing['call_to_action']}")
    
    print(f"\n🔍 SEO KEYWORDS:")
    print(f"   {', '.join(watch_marketing['seo_keywords'])}")
else:
    print("❌ Error generando marketing para relojería")

## 🔗 7. Test de Flujo Completo Integrado
Simulamos el flujo completo: Marketing → Traducción → Resultado Final

In [ ]:
# Función de flujo completo
async def complete_multilingual_flow(product: ProductSpecs, languages: List[TargetLanguage], brand_name: str):
    """Ejecuta el flujo completo de marketing multiidioma"""
    
    result = {
        "product_name": product.name,
        "brand_name": brand_name,
        "processing_steps": [],
        "original_marketing": None,
        "translations": [],
        "summary": {}
    }
    
    start_time = datetime.now()
    
    # Paso 1: Generar marketing
    print("🎨 PASO 1: Generando contenido de marketing...")
    result["processing_steps"].append("Iniciando generación de marketing")
    
    marketing = await generate_marketing_content(product)
    
    if not marketing:
        result["processing_steps"].append("❌ Error en generación de marketing")
        return result
    
    result["original_marketing"] = marketing
    result["processing_steps"].append("✅ Marketing generado exitosamente")
    print(f"   ✅ Marketing generado: {len(marketing['description'].split())} palabras")
    
    # Paso 2: Traducir a múltiples idiomas
    print(f"\n🌍 PASO 2: Traduciendo a {len(languages)} idiomas...")
    result["processing_steps"].append(f"Iniciando traducción a {len(languages)} idiomas")
    
    for i, lang in enumerate(languages, 1):
        print(f"   ⏳ ({i}/{len(languages)}) Traduciendo a {lang.value}...")
        
        translation = await translate_content(
            text=marketing['description'],
            target_language=lang,
            brand_name=brand_name
        )
        
        if translation:
            result["translations"].append(translation)
            result["processing_steps"].append(f"✅ Traducción a {lang.value} completada")
            print(f"      ✅ Completado")
        else:
            result["processing_steps"].append(f"❌ Error traduciendo a {lang.value}")
            print(f"      ❌ Error")
    
    # Paso 3: Generar resumen
    end_time = datetime.now()
    processing_time = (end_time - start_time).total_seconds()
    
    result["summary"] = {
        "total_processing_time": round(processing_time, 2),
        "languages_requested": len(languages),
        "languages_completed": len(result["translations"]),
        "success_rate": round((len(result["translations"]) / len(languages)) * 100, 1),
        "original_word_count": len(marketing['description'].split()),
        "completion_timestamp": end_time.isoformat()
    }
    
    result["processing_steps"].append("🎉 Flujo completo terminado")
    
    return result

print("✅ Función de flujo completo lista")

In [ ]:
# Ejecutar flujo completo con el reloj
target_languages = [
    TargetLanguage.ENGLISH,
    TargetLanguage.FRENCH,
    TargetLanguage.ITALIAN
]

print("🚀 EJECUTANDO FLUJO COMPLETO")
print("="*50)
print(f"📦 Producto: {watch_product.name}")
print(f"🌍 Idiomas: {[lang.value for lang in target_languages]}")
print(f"🏢 Marca: Relojería Barcelona Premium")
print("\n⏳ Iniciando procesamiento...")

complete_result = await complete_multilingual_flow(
    product=watch_product,
    languages=target_languages,
    brand_name="Relojería Barcelona Premium"
)

In [ ]:
# Mostrar resultado completo
if complete_result and complete_result["original_marketing"]:
    print("\n" + "="*80)
    print("🎉 RESULTADO FINAL - MARKETING MULTIIDIOMA COMPLETO")
    print("="*80)
    
    # Información del producto
    print(f"\n📦 PRODUCTO: {complete_result['product_name']}")
    print(f"🏢 MARCA: {complete_result['brand_name']}")
    
    # Marketing original
    marketing = complete_result["original_marketing"]
    print(f"\n📝 MARKETING ORIGINAL (Español):")
    print(f"   🎯 {marketing['title']}")
    print(f"   📄 {marketing['description']}")
    
    # Traducciones
    print(f"\n🌍 TRADUCCIONES:")
    for translation in complete_result["translations"]:
        lang_names = {
            "en": "🇺🇸 Inglés",
            "fr": "🇫🇷 Francés",
            "it": "🇮🇹 Italiano",
            "de": "🇩🇪 Alemán",
            "pt": "🇵🇹 Portugués",
            "ca": "🏴 Catalán"
        }
        
        lang_name = lang_names.get(translation['language'], f"🌍 {translation['language']}")
        print(f"\n   {lang_name}:")
        print(f"      {translation['translated_text']}")
        
        if translation['cultural_notes']:
            print(f"      📌 Adaptaciones: {'; '.join(translation['cultural_notes'])}")
    
    # Estadísticas finales
    summary = complete_result["summary"]
    print(f"\n📊 ESTADÍSTICAS FINALES:")
    print(f"   ⏱️ Tiempo total de procesamiento: {summary['total_processing_time']} segundos")
    print(f"   🌍 Idiomas solicitados: {summary['languages_requested']}")
    print(f"   ✅ Idiomas completados: {summary['languages_completed']}")
    print(f"   📈 Tasa de éxito: {summary['success_rate']}%")
    print(f"   📝 Palabras en descripción original: {summary['original_word_count']}")
    
    # Procesamiento step by step
    print(f"\n🔄 LOG DE PROCESAMIENTO:")
    for i, step in enumerate(complete_result["processing_steps"], 1):
        print(f"   {i}. {step}")
    
    print(f"\n🎯 CASOS DE USO DEMOSTRADOS:")
    print(f"   ✅ Generación automática de marketing especializado")
    print(f"   ✅ Traducción cultural para mercados internacionales")
    print(f"   ✅ Flujo completo integrado desde especificaciones")
    print(f"   ✅ Escalabilidad para múltiples productos e idiomas")
    
else:
    print("❌ Error en el flujo completo")

## ✅ 8. Resumen de Pruebas y Conclusiones

In [ ]:
# Resumen final de todas las pruebas
print("📋 RESUMEN DE PRUEBAS REALIZADAS")
print("="*50)

tests_completed = []

# Verificar qué tests se completaron
if 'OPENAI_API_KEY' in locals() and OPENAI_API_KEY:
    tests_completed.append("✅ Configuración de OpenAI API")
else:
    tests_completed.append("❌ Configuración de OpenAI API")

if 'marketing_result' in locals() and marketing_result:
    tests_completed.append("✅ API 1: Generador de Marketing (Suite Hotel)")
else:
    tests_completed.append("❌ API 1: Generador de Marketing (Suite Hotel)")

if 'translations' in locals() and translations:
    tests_completed.append(f"✅ API 2: Traductor Especializado ({len(translations)} idiomas)")
else:
    tests_completed.append("❌ API 2: Traductor Especializado")

if 'watch_marketing' in locals() and watch_marketing:
    tests_completed.append("✅ Especialización: Marketing de Relojería")
else:
    tests_completed.append("❌ Especialización: Marketing de Relojería")

if 'complete_result' in locals() and complete_result.get('original_marketing'):
    tests_completed.append("✅ Flujo Completo Integrado")
else:
    tests_completed.append("❌ Flujo Completo Integrado")

# Mostrar resultados
for test in tests_completed:
    print(f"   {test}")

# Calcular tasa de éxito
successful_tests = sum(1 for test in tests_completed if test.startswith("✅"))
total_tests = len(tests_completed)
success_rate = (successful_tests / total_tests) * 100

print(f"\n📊 ESTADÍSTICAS GENERALES:")
print(f"   Tests exitosos: {successful_tests}/{total_tests}")
print(f"   Tasa de éxito: {success_rate:.1f}%")

if success_rate >= 80:
    print(f"\n🎉 SISTEMA LISTO PARA PRODUCCIÓN")
    print(f"   Las APIs están funcionando correctamente")
    print(f"   Se puede proceder con la demostración en clase")
elif success_rate >= 60:
    print(f"\n⚠️ SISTEMA PARCIALMENTE FUNCIONAL")
    print(f"   Algunos componentes necesitan ajustes")
    print(f"   Revisar tests fallidos antes de la clase")
else:
    print(f"\n❌ SISTEMA REQUIERE ATENCIÓN")
    print(f"   Varios componentes presentan problemas")
    print(f"   Revisar configuración y conexiones")

print(f"\n🎯 PRÓXIMOS PASOS PARA LA CLASE:")
print(f"   1. Verificar que todas las dependencias estén instaladas")
print(f"   2. Configurar API keys de OpenAI en cada máquina")
print(f"   3. Ejecutar este notebook como demo en vivo")
print(f"   4. Preparar ejemplos adicionales según audiencia")
print(f"   5. Tener scripts de respaldo en caso de problemas de red")

## 🚀 9. Preparación para la Clase

### ✅ **Lista de Verificación Pre-Clase**

**Configuración Técnica:**
- [ ] Entorno virtual creado y activado
- [ ] Todas las dependencias instaladas
- [ ] API key de OpenAI configurada
- [ ] Notebook ejecutándose sin errores
- [ ] Conexión a internet estable

**Contenido de Demo:**
- [ ] Ejemplo de hotel funcionando
- [ ] Ejemplo de relojería funcionando  
- [ ] Traducciones a múltiples idiomas
- [ ] Flujo completo integrado
- [ ] Métricas y estadísticas visibles

**Material de Respaldo:**
- [ ] Screenshots de resultados exitosos
- [ ] Ejemplos pre-generados por si falla la API
- [ ] Scripts alternativos con datos mock

### 🎯 **Flujo Sugerido para la Demo en Clase**

1. **Introducción (5 min)**: Mostrar este notebook y explicar el objetivo
2. **API 1 Demo (10 min)**: Ejecutar generación de marketing en vivo
3. **API 2 Demo (10 min)**: Traducir el contenido generado
4. **Caso Alternativo (10 min)**: Relojería para mostrar especialización
5. **Flujo Integrado (10 min)**: Demostrar el pipeline completo
6. **Q&A y Hands-on (15 min)**: Que participantes prueben con sus casos

### 💡 **Tips para el Instructor**

- **Ejecutar este notebook** antes de cada clase para verificar funcionamiento
- **Tener ejemplos preparados** por si hay problemas de conectividad
- **Explicar el valor de negocio** en cada paso, no solo la tecnología
- **Mostrar diferencias** vs herramientas no-code durante la demo
- **Enfatizar personalización** y control total sobre el proceso